## Relu
- problem of sigmoid
    - 기존 neural network 가 학습하는 방법 : ground truth data와 output data의 차이(<font color="red">loss</font>)를 미분한것(<font color="red">gradient = 기울기</font>)을 back propagation 함
    - 깊은 네트워크에서 매우 작은 gradient 값들이(0에 가까움) 곱해지면 <font color="blue"> __기울기 소실(vanishing gradient)__ </font> 문제 발생
- why relu
    - $f(x)= max(0,x)$
    - 0보다 클때 gradient가 1임, 0보다 작을때는 gradient가 0이므로 gradient가 전달되지 않음
    - 0보다 작은 graident 문제 해결, 0.001 정도곱해줌 : leaky relu(tf.keras.layers)

In [5]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
import os
from time import time

C:\Users\alaco\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

In [15]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1] axis=3 이라고 해도됨
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1] [batch_size,height,width,channel]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10] , onehotencoding
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

In [8]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

In [9]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

In [10]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.RandomNormal()

        self.model = tf.keras.Sequential() #list자료구조 타입
        self.model.add(flatten()) #fc layer 쓸꺼기 때문에 바꿔줌

        for i in range(2):
            self.model.add(dense(256, weight_init))
            self.model.add(relu())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):
        x = self.model(x)

        return x

In [11]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.RandomNormal()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(2) :
        model.add(dense(256, weight_init))
        model.add(relu())

    model.add(dense(label_dim, weight_init))

    return model

In [12]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices ((train_x, train_y)).\
    shuffle(buffer_size=100000).\ #6만보다 큰 숫자면 됨
    prefetch(buffer_size=batch_size).\ #미리 메모리에 batch size 만큼 옮겨놔라
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

In [13]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_relu'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

In [14]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):                
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)

                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 0.4445, train_loss: 2.14571881, train_accuracy: 0.4531, test_Accuracy: 0.2498
Epoch: [ 0] [    1/  468] time: 0.5278, train_loss: 2.12704897, train_accuracy: 0.4766, test_Accuracy: 0.3947
Epoch: [ 0] [    2/  468] time: 0.6118, train_loss: 2.04538941, train_accuracy: 0.6016, test_Accuracy: 0.5518
Epoch: [ 0] [    3/  468] time: 0.7036, train_loss: 2.01838613, train_accuracy: 0.5859, test_Accuracy: 0.6345
Epoch: [ 0] [    4/  468] time: 0.7820, train_loss: 1.86895537, train_accuracy: 0.6484, test_Accuracy: 0.6438
Epoch: [ 0] [    5/  468] time: 0.8591, train_loss: 1.87096000, train_accuracy: 0.5781, test_Accuracy: 0.6476
Epoch: [ 0] [    6/  468] time: 0.9365, train_loss: 1.72059166, train_accuracy: 0.6484, test_Accuracy: 0.6601
Epoch: [ 0] [    7/  468] time: 1.0300, train_loss: 1.66095865, train_accuracy: 0.7188, test_Accuracy: 0.7168
Epoch: [ 0] [    8/  468] time: 1.1123,

Epoch: [ 0] [   76/  468] time: 7.1037, train_loss: 0.36098897, train_accuracy: 0.8984, test_Accuracy: 0.9103
Epoch: [ 0] [   77/  468] time: 7.1888, train_loss: 0.54272014, train_accuracy: 0.8594, test_Accuracy: 0.9117
Epoch: [ 0] [   78/  468] time: 7.2749, train_loss: 0.32324570, train_accuracy: 0.9219, test_Accuracy: 0.9100
Epoch: [ 0] [   79/  468] time: 7.3609, train_loss: 0.24581666, train_accuracy: 0.9453, test_Accuracy: 0.9069
Epoch: [ 0] [   80/  468] time: 7.4571, train_loss: 0.32327473, train_accuracy: 0.9219, test_Accuracy: 0.9032
Epoch: [ 0] [   81/  468] time: 7.5419, train_loss: 0.22447452, train_accuracy: 0.9531, test_Accuracy: 0.9035
Epoch: [ 0] [   82/  468] time: 7.6295, train_loss: 0.30418780, train_accuracy: 0.9375, test_Accuracy: 0.9081
Epoch: [ 0] [   83/  468] time: 7.7158, train_loss: 0.28690633, train_accuracy: 0.9141, test_Accuracy: 0.9096
Epoch: [ 0] [   84/  468] time: 7.8118, train_loss: 0.30703035, train_accuracy: 0.9062, test_Accuracy: 0.9074
Epoch: [ 0

Epoch: [ 0] [  152/  468] time: 13.9102, train_loss: 0.24195330, train_accuracy: 0.9375, test_Accuracy: 0.9347
Epoch: [ 0] [  153/  468] time: 14.0002, train_loss: 0.32211190, train_accuracy: 0.9297, test_Accuracy: 0.9344
Epoch: [ 0] [  154/  468] time: 14.0870, train_loss: 0.10834455, train_accuracy: 0.9688, test_Accuracy: 0.9351
Epoch: [ 0] [  155/  468] time: 14.1734, train_loss: 0.21725833, train_accuracy: 0.9453, test_Accuracy: 0.9337
Epoch: [ 0] [  156/  468] time: 14.2559, train_loss: 0.31738555, train_accuracy: 0.9219, test_Accuracy: 0.9325
Epoch: [ 0] [  157/  468] time: 14.3479, train_loss: 0.27054781, train_accuracy: 0.9062, test_Accuracy: 0.9317
Epoch: [ 0] [  158/  468] time: 14.4338, train_loss: 0.12675634, train_accuracy: 0.9766, test_Accuracy: 0.9310
Epoch: [ 0] [  159/  468] time: 14.5185, train_loss: 0.33270052, train_accuracy: 0.9141, test_Accuracy: 0.9307
Epoch: [ 0] [  160/  468] time: 14.5991, train_loss: 0.28144071, train_accuracy: 0.9375, test_Accuracy: 0.9319
E

Epoch: [ 0] [  228/  468] time: 20.6462, train_loss: 0.18131350, train_accuracy: 0.9609, test_Accuracy: 0.9406
Epoch: [ 0] [  229/  468] time: 20.7304, train_loss: 0.20886207, train_accuracy: 0.9531, test_Accuracy: 0.9420
Epoch: [ 0] [  230/  468] time: 20.8250, train_loss: 0.19170734, train_accuracy: 0.9531, test_Accuracy: 0.9431
Epoch: [ 0] [  231/  468] time: 20.9088, train_loss: 0.22500999, train_accuracy: 0.9219, test_Accuracy: 0.9443
Epoch: [ 0] [  232/  468] time: 20.9927, train_loss: 0.15955885, train_accuracy: 0.9453, test_Accuracy: 0.9452
Epoch: [ 0] [  233/  468] time: 21.0769, train_loss: 0.17604883, train_accuracy: 0.9453, test_Accuracy: 0.9449
Epoch: [ 0] [  234/  468] time: 21.1677, train_loss: 0.27203310, train_accuracy: 0.8906, test_Accuracy: 0.9447
Epoch: [ 0] [  235/  468] time: 21.2524, train_loss: 0.12436974, train_accuracy: 0.9609, test_Accuracy: 0.9416
Epoch: [ 0] [  236/  468] time: 21.3426, train_loss: 0.20187208, train_accuracy: 0.9609, test_Accuracy: 0.9399
E

Epoch: [ 0] [  303/  468] time: 27.1810, train_loss: 0.12479360, train_accuracy: 0.9453, test_Accuracy: 0.9519
Epoch: [ 0] [  304/  468] time: 27.2663, train_loss: 0.18623096, train_accuracy: 0.9375, test_Accuracy: 0.9501
Epoch: [ 0] [  305/  468] time: 27.3641, train_loss: 0.16999221, train_accuracy: 0.9453, test_Accuracy: 0.9488
Epoch: [ 0] [  306/  468] time: 27.4651, train_loss: 0.11217961, train_accuracy: 0.9766, test_Accuracy: 0.9475
Epoch: [ 0] [  307/  468] time: 27.5715, train_loss: 0.13273831, train_accuracy: 0.9453, test_Accuracy: 0.9470
Epoch: [ 0] [  308/  468] time: 27.6592, train_loss: 0.20128766, train_accuracy: 0.9375, test_Accuracy: 0.9472
Epoch: [ 0] [  309/  468] time: 27.7519, train_loss: 0.12001313, train_accuracy: 0.9531, test_Accuracy: 0.9483
Epoch: [ 0] [  310/  468] time: 27.8398, train_loss: 0.23739421, train_accuracy: 0.9062, test_Accuracy: 0.9491
Epoch: [ 0] [  311/  468] time: 27.9265, train_loss: 0.09269812, train_accuracy: 0.9688, test_Accuracy: 0.9485
E

Epoch: [ 0] [  379/  468] time: 34.5586, train_loss: 0.17594352, train_accuracy: 0.9609, test_Accuracy: 0.9546
Epoch: [ 0] [  380/  468] time: 34.6454, train_loss: 0.15431581, train_accuracy: 0.9531, test_Accuracy: 0.9551
Epoch: [ 0] [  381/  468] time: 34.7411, train_loss: 0.10308652, train_accuracy: 0.9844, test_Accuracy: 0.9551
Epoch: [ 0] [  382/  468] time: 34.8423, train_loss: 0.20877713, train_accuracy: 0.9375, test_Accuracy: 0.9534
Epoch: [ 0] [  383/  468] time: 34.9384, train_loss: 0.07827712, train_accuracy: 0.9766, test_Accuracy: 0.9530
Epoch: [ 0] [  384/  468] time: 35.0418, train_loss: 0.15726833, train_accuracy: 0.9531, test_Accuracy: 0.9506
Epoch: [ 0] [  385/  468] time: 35.1360, train_loss: 0.08146057, train_accuracy: 0.9766, test_Accuracy: 0.9486
Epoch: [ 0] [  386/  468] time: 35.2289, train_loss: 0.16051309, train_accuracy: 0.9297, test_Accuracy: 0.9474
Epoch: [ 0] [  387/  468] time: 35.3244, train_loss: 0.21094491, train_accuracy: 0.9219, test_Accuracy: 0.9493
E

Epoch: [ 0] [  455/  468] time: 42.0032, train_loss: 0.14331403, train_accuracy: 0.9688, test_Accuracy: 0.9578
Epoch: [ 0] [  456/  468] time: 42.0949, train_loss: 0.10378358, train_accuracy: 0.9688, test_Accuracy: 0.9595
Epoch: [ 0] [  457/  468] time: 42.1933, train_loss: 0.14168753, train_accuracy: 0.9609, test_Accuracy: 0.9595
Epoch: [ 0] [  458/  468] time: 42.3035, train_loss: 0.17089719, train_accuracy: 0.9531, test_Accuracy: 0.9601
Epoch: [ 0] [  459/  468] time: 42.4107, train_loss: 0.09488395, train_accuracy: 0.9844, test_Accuracy: 0.9602
Epoch: [ 0] [  460/  468] time: 42.5102, train_loss: 0.13562462, train_accuracy: 0.9531, test_Accuracy: 0.9600
Epoch: [ 0] [  461/  468] time: 42.6102, train_loss: 0.05654427, train_accuracy: 0.9844, test_Accuracy: 0.9581
Epoch: [ 0] [  462/  468] time: 42.7017, train_loss: 0.18681313, train_accuracy: 0.9531, test_Accuracy: 0.9570
Epoch: [ 0] [  463/  468] time: 42.7896, train_loss: 0.11837553, train_accuracy: 0.9609, test_Accuracy: 0.9546
E

## Weight Initialization
- xavier initialization(glorot)
    - global minima 에 수렴하지 않고 local minima 나 saddle point 에 수렴할수있음
    - 평균 : 0, 분산 : 2/channel_in + channel_out 
- he initialization(relu)
    - 평균 : 0, 분산 : 4/channel_in + channel_out 


#### glorot

In [16]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.9.1


In [17]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

In [18]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

In [19]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

In [20]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

In [21]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform() #he_uniform

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(2):
            self.model.add(dense(256, weight_init))
            self.model.add(relu())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):

        x = self.model(x)

        return x

In [22]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

In [23]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_xavier'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

In [24]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):                
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)

                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 0.3114, train_loss: 2.16964340, train_accuracy: 0.3047, test_Accuracy: 0.2099
Epoch: [ 0] [    1/  468] time: 0.3920, train_loss: 2.10007000, train_accuracy: 0.4375, test_Accuracy: 0.3504
Epoch: [ 0] [    2/  468] time: 0.4736, train_loss: 2.04121590, train_accuracy: 0.5312, test_Accuracy: 0.4562
Epoch: [ 0] [    3/  468] time: 0.5582, train_loss: 1.95338249, train_accuracy: 0.5312, test_Accuracy: 0.5206
Epoch: [ 0] [    4/  468] time: 0.6374, train_loss: 1.94407964, train_accuracy: 0.5469, test_Accuracy: 0.5900
Epoch: [ 0] [    5/  468] time: 0.7165, train_loss: 1.76370585, train_accuracy: 0.7578, test_Accuracy: 0.6431
Epoch: [ 0] [    6/  468] time: 0.7984, train_loss: 1.66720784, train_accuracy: 0.7188, test_Accuracy: 0.6705
Epoch: [ 0] [    7/  468] time: 0.8800, train_loss: 1.50247526, train_accuracy: 0.7188, test_Accuracy: 0.6756
Epoch: [ 0] [    8/  468] time: 0.9583,

Epoch: [ 0] [   75/  468] time: 7.0180, train_loss: 0.34813136, train_accuracy: 0.8906, test_Accuracy: 0.9075
Epoch: [ 0] [   76/  468] time: 7.1152, train_loss: 0.26197571, train_accuracy: 0.9062, test_Accuracy: 0.9085
Epoch: [ 0] [   77/  468] time: 7.2187, train_loss: 0.38726619, train_accuracy: 0.8984, test_Accuracy: 0.9116
Epoch: [ 0] [   78/  468] time: 7.3388, train_loss: 0.23187074, train_accuracy: 0.9375, test_Accuracy: 0.9117
Epoch: [ 0] [   79/  468] time: 7.4313, train_loss: 0.26514304, train_accuracy: 0.9219, test_Accuracy: 0.9131
Epoch: [ 0] [   80/  468] time: 7.5373, train_loss: 0.28905952, train_accuracy: 0.9297, test_Accuracy: 0.9150
Epoch: [ 0] [   81/  468] time: 7.6277, train_loss: 0.33556575, train_accuracy: 0.8984, test_Accuracy: 0.9138
Epoch: [ 0] [   82/  468] time: 7.7184, train_loss: 0.34191287, train_accuracy: 0.8906, test_Accuracy: 0.9175
Epoch: [ 0] [   83/  468] time: 7.8020, train_loss: 0.14008957, train_accuracy: 0.9453, test_Accuracy: 0.9135
Epoch: [ 0

Epoch: [ 0] [  150/  468] time: 13.6750, train_loss: 0.29787090, train_accuracy: 0.9297, test_Accuracy: 0.9312
Epoch: [ 0] [  151/  468] time: 13.7599, train_loss: 0.26511765, train_accuracy: 0.8906, test_Accuracy: 0.9303
Epoch: [ 0] [  152/  468] time: 13.8476, train_loss: 0.23826319, train_accuracy: 0.9453, test_Accuracy: 0.9294
Epoch: [ 0] [  153/  468] time: 13.9544, train_loss: 0.15057428, train_accuracy: 0.9375, test_Accuracy: 0.9273
Epoch: [ 0] [  154/  468] time: 14.0457, train_loss: 0.18101251, train_accuracy: 0.9219, test_Accuracy: 0.9285
Epoch: [ 0] [  155/  468] time: 14.1358, train_loss: 0.28647888, train_accuracy: 0.8984, test_Accuracy: 0.9304
Epoch: [ 0] [  156/  468] time: 14.2227, train_loss: 0.33199555, train_accuracy: 0.9141, test_Accuracy: 0.9311
Epoch: [ 0] [  157/  468] time: 14.3113, train_loss: 0.22674103, train_accuracy: 0.9219, test_Accuracy: 0.9306
Epoch: [ 0] [  158/  468] time: 14.3972, train_loss: 0.42809626, train_accuracy: 0.8828, test_Accuracy: 0.9298
E

Epoch: [ 0] [  224/  468] time: 20.2700, train_loss: 0.19549818, train_accuracy: 0.9531, test_Accuracy: 0.9466
Epoch: [ 0] [  225/  468] time: 20.3563, train_loss: 0.26090431, train_accuracy: 0.9297, test_Accuracy: 0.9460
Epoch: [ 0] [  226/  468] time: 20.4400, train_loss: 0.17932378, train_accuracy: 0.9453, test_Accuracy: 0.9449
Epoch: [ 0] [  227/  468] time: 20.5281, train_loss: 0.28045624, train_accuracy: 0.9531, test_Accuracy: 0.9445
Epoch: [ 0] [  228/  468] time: 20.6129, train_loss: 0.26883158, train_accuracy: 0.9219, test_Accuracy: 0.9445
Epoch: [ 0] [  229/  468] time: 20.6991, train_loss: 0.19157000, train_accuracy: 0.9297, test_Accuracy: 0.9446
Epoch: [ 0] [  230/  468] time: 20.7954, train_loss: 0.12542489, train_accuracy: 0.9688, test_Accuracy: 0.9441
Epoch: [ 0] [  231/  468] time: 20.8826, train_loss: 0.20288078, train_accuracy: 0.9453, test_Accuracy: 0.9461
Epoch: [ 0] [  232/  468] time: 20.9915, train_loss: 0.19653532, train_accuracy: 0.9219, test_Accuracy: 0.9449
E

Epoch: [ 0] [  299/  468] time: 27.0432, train_loss: 0.20422450, train_accuracy: 0.9453, test_Accuracy: 0.9382
Epoch: [ 0] [  300/  468] time: 27.1437, train_loss: 0.22831690, train_accuracy: 0.9453, test_Accuracy: 0.9398
Epoch: [ 0] [  301/  468] time: 27.2332, train_loss: 0.27987617, train_accuracy: 0.9297, test_Accuracy: 0.9433
Epoch: [ 0] [  302/  468] time: 27.3234, train_loss: 0.14782877, train_accuracy: 0.9688, test_Accuracy: 0.9482
Epoch: [ 0] [  303/  468] time: 27.4095, train_loss: 0.22593698, train_accuracy: 0.9531, test_Accuracy: 0.9502
Epoch: [ 0] [  304/  468] time: 27.4989, train_loss: 0.13474095, train_accuracy: 0.9531, test_Accuracy: 0.9498
Epoch: [ 0] [  305/  468] time: 27.5871, train_loss: 0.12245242, train_accuracy: 0.9609, test_Accuracy: 0.9428
Epoch: [ 0] [  306/  468] time: 27.6765, train_loss: 0.18517147, train_accuracy: 0.9453, test_Accuracy: 0.9399
Epoch: [ 0] [  307/  468] time: 27.7644, train_loss: 0.26540431, train_accuracy: 0.9062, test_Accuracy: 0.9441
E

Epoch: [ 0] [  374/  468] time: 34.2731, train_loss: 0.23515901, train_accuracy: 0.9531, test_Accuracy: 0.9493
Epoch: [ 0] [  375/  468] time: 34.3663, train_loss: 0.12821347, train_accuracy: 0.9844, test_Accuracy: 0.9488
Epoch: [ 0] [  376/  468] time: 34.4569, train_loss: 0.11363638, train_accuracy: 0.9766, test_Accuracy: 0.9505
Epoch: [ 0] [  377/  468] time: 34.5573, train_loss: 0.08690359, train_accuracy: 0.9688, test_Accuracy: 0.9537
Epoch: [ 0] [  378/  468] time: 34.6442, train_loss: 0.11554971, train_accuracy: 0.9688, test_Accuracy: 0.9540
Epoch: [ 0] [  379/  468] time: 34.7359, train_loss: 0.11775587, train_accuracy: 0.9609, test_Accuracy: 0.9546
Epoch: [ 0] [  380/  468] time: 34.8297, train_loss: 0.11381650, train_accuracy: 0.9609, test_Accuracy: 0.9537
Epoch: [ 0] [  381/  468] time: 34.9643, train_loss: 0.10645853, train_accuracy: 0.9766, test_Accuracy: 0.9536
Epoch: [ 0] [  382/  468] time: 35.0751, train_loss: 0.11181250, train_accuracy: 0.9609, test_Accuracy: 0.9528
E

Epoch: [ 0] [  448/  468] time: 42.8759, train_loss: 0.13036913, train_accuracy: 0.9609, test_Accuracy: 0.9582
Epoch: [ 0] [  449/  468] time: 43.0997, train_loss: 0.08954701, train_accuracy: 0.9922, test_Accuracy: 0.9584
Epoch: [ 0] [  450/  468] time: 43.2742, train_loss: 0.07779467, train_accuracy: 0.9766, test_Accuracy: 0.9592
Epoch: [ 0] [  451/  468] time: 43.4238, train_loss: 0.07202603, train_accuracy: 0.9766, test_Accuracy: 0.9588
Epoch: [ 0] [  452/  468] time: 43.5619, train_loss: 0.09307672, train_accuracy: 0.9766, test_Accuracy: 0.9592
Epoch: [ 0] [  453/  468] time: 43.6877, train_loss: 0.10826641, train_accuracy: 0.9688, test_Accuracy: 0.9604
Epoch: [ 0] [  454/  468] time: 43.8040, train_loss: 0.08370051, train_accuracy: 0.9766, test_Accuracy: 0.9619
Epoch: [ 0] [  455/  468] time: 43.9213, train_loss: 0.07129952, train_accuracy: 0.9766, test_Accuracy: 0.9620
Epoch: [ 0] [  456/  468] time: 44.0363, train_loss: 0.06399879, train_accuracy: 0.9922, test_Accuracy: 0.9601
E

## Dropout
- dropout 
    - 모든 뉴런혹은 노드를 학습하는게 아니라 일부만 학습함
    - 몇개 제외 : 랜덤
    - regularization


In [ ]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

def dropout(rate) :
    return tf.keras.layers.Dropout(rate)

In [ ]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(4):
            self.model.add(dense(512, weight_init))
            self.model.add(relu())
            self.model.add(dropout(rate=0.5)) #50%의 뉴런 제외하겟다는 뜻

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):

        x = self.model(x)

        return x

In [ ]:
def loss_fn(model, images, labels):
    logits = model(images, training=True) #true : dropout 사용하겟다는뜻
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

## Batch Normalization
- batch normalization
     - IF 고양이 이미지를 받아서 고양이인지 아닌지 확인하는 네트워크
     - layer 를지나면서 input distribution 모양 변화함
     - internal covariate shift 를 막자는 의도
     - layer input distribution 을 계속 normalization 함

In [ ]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

def batch_norm() :
    return tf.keras.layers.BatchNormalization()

In [ ]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(4):
            self.model.add(dense(512, weight_init))
            self.model.add(batch_norm())
            self.model.add(relu())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):

        x = self.model(x)

        return x

dropout 은 relu activation function 다음에 썼는데  
batch nomarlization 은 relu 전에 씀  
why ? layer- norm- activation 순서로 혹은 norm-activation-layer